In [10]:
import datetime

In [12]:
MODEL_NAME = "fulltime_draw_outcome"
MODEL_VERSION = 1.0

In [13]:
# UPDATING THE GITHUB OF THE DATASETS
MODEL_TIME = datetime.datetime.now().strftime("%d-%M-%Y_%T")
!git add -A
!git commit -m "updates: {MODEL_NAME}_v{MODEL_VERSION}: bot commit - {MODEL_TIME}"
!git push

[main 761fb77] updates: fulltime_draw_outcome_v1.0: bot commit - 27-17-2024_11:17:08
 4 files changed, 15144 insertions(+), 24 deletions(-)
 create mode 100644 relative_datasets/cleaned/fulltime_draw_outcome_v1.0-test-set.csv
 create mode 100644 relative_datasets/cleaned/fulltime_draw_outcome_v1.0-train-set.csv


Enumerating objects: 15, done.
Counting objects: 100% (15/15), done.
Delta compression using up to 2 threads
Compressing objects: 100% (9/9), done.
Writing objects: 100% (9/9), 108.63 KiB | 783.00 KiB/s, done.
Total 9 (delta 5), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (5/5), completed with 5 local objects.
To https://github.com/lebyanelm/neural-trained-models.git
   eee2801..761fb77  main -> main
